In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import openpyxl
import os
import pickle

scope = ["https://www.googleapis.com/auth/youtube.force-ssl"] #define how I'll use the YouTube API

def youtube_authenticate(): #authenticate to YouTube API https://developers.google.com/youtube/v3
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = r"..."
    creds = None 
    if os.path.exists("token.pickle"): #create string representation of my credentials to reference later
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, scope)
            creds = flow.run_local_server(port=0)
    with open("token.pickle", "wb") as token: #reuse credentials 
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds) #will use to get API response for videos

youtube = youtube_authenticate()

def get_video_id_by_url(url): #parse URL query and get video ID
    parsed_url = p.urlparse(url)
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")
    
def get_video_details(youtube, **kwargs): #pass return from youtube_authenticate and keywords from video ID
    return youtube.videos().list(part="snippet,contentDetails,statistics", **kwargs).execute() #will need info from these parts of the API 

detail_list = [] #this list will be used to append to Excel worksheet

def list_video_infos(video_response):
    items = video_response.get("items")[0]
    snippet = items["snippet"]
    statistics = items["statistics"]
    title = snippet["title"]
    comment_count = statistics["commentCount"]
    like_count = statistics["likeCount"]
    view_count = statistics["viewCount"]
    info_list = [title, comment_count, like_count, view_count]
    detail_list.append(info_list) #used to create list of lists

wb = openpyxl.load_workbook(r"...", read_only=False, keep_vba=False) #open doc as writeable, and xlsx does not contain VBA
ws = wb.active
link_list = [ws.cell(row=i, column=2).value for i in range(2, 51)]

num = 0 #will use for tracking execution

for link in link_list:
    video_url = link
    video_id = get_video_id_by_url(video_url) #parse URL for video ID
    response = get_video_details(youtube, id=video_id) #retrieve video details
    list_video_infos(response) #append video details to list for addition to Excel worksheet
    num += 1

for video in detail_list:
    ws.append(video) 

print(f"Of the {len(link_list)} links to retrieve data from, {num} retrievals were successful")

wb.save("YouTube Worksheet.xlsx")

